# Question answering with LLM

In [ ]:
#|default_exp qa.ablation

In [ ]:
#|hide
from fastcore.test import *
from nbdev.showdoc import *

In [ ]:
#|hide
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
#|export
import magentic
from pydantic import BaseModel, Field

from bellem.logging import get_logger

log = get_logger(__name__)

In [ ]:
#|export

USER_PROMPT = """The context information is provided below.
---------------------
{context}
---------------------
Given the context information and not prior knowledge, answer the question.
{question}
"""

In [ ]:
#|export

SYSTEM_PROMPT_STANDARD = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge."""

class QuestionAnsweringResultStandard(BaseModel):
    """Data model for answering the question."""

    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_STANDARD), 
    magentic.UserMessage(USER_PROMPT),
)
def answer_question_standard(
    context: str,
    question: str,
) -> QuestionAnsweringResultStandard: ...

In [ ]:
#|export

SYSTEM_PROMPT_COT = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your response."""

class QuestionAnsweringResultCOT(BaseModel):
    """Data model for answering the question."""

    reasoning: str = Field(description="Step-by-step reasoning for the answer.")
    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_COT), 
    magentic.UserMessage(USER_PROMPT),
)
def answer_question_cot(
    context: str,
    question: str,
) -> QuestionAnsweringResultCOT: ...

In [ ]:
#|export

SYSTEM_PROMPT_COT_FS = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your response.

# Example
Context: "Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011."

Question: "Who is the current mayor of the city Glenhis Hernández was born?"

Reasoning:
1. The context states that Glenhis Hernández was born in Havana.
2. It also mentions that the current mayor (President of the People's Power Provincial Assembly) of Havana is Marta Hernández Romero.
3. Marta Hernández Romero has been the mayor since her election on March 5, 2011.

Answer: "Marta Hernández Romero"
"""

class QuestionAnsweringResultCOTFS(BaseModel):
    """Data model for answering the question."""

    reasoning: str = Field(description="Step-by-step reasoning for the answer.")
    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_COT_FS), 
    magentic.UserMessage(USER_PROMPT),
)
def answer_question_cot_fs(
    context: str,
    question: str,
) -> QuestionAnsweringResultCOTFS: ...

In [ ]:
#|export

SYSTEM_PROMPT_CTE = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on the context information given, without drawing on prior knowledge.

Before answering the question, first, you extract relevant entity-relation-entity triplets from the context. Then, you answer the question based on the triplets. For instance, 

# Example
Context: "Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011."

Question: "Who is the current mayor of city Glenhis Hernández was born?"

Triplets: 
Glenhis Hernández | birth place | Havana
Marta Hernández Romero | serves as | mayor of Havana

Answer: "Marta Hernández Romero"
"""

class _QuestionAnsweringResultCTE(BaseModel):
    """Data model for answering the question."""

    triplets: list[str] = Field(description="A list of entity-relation-entity triplets extracted from the context.")
    answer: str = Field(description="The answer to the question in 2-4 words.")

class QuestionAnsweringResultCTE(_QuestionAnsweringResultCTE):
    reasoning: str = ""

@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_CTE), 
    magentic.UserMessage(USER_PROMPT),
)
def _answer_question_cte(
    context: str,
    question: str,
) -> _QuestionAnsweringResultCTE: ...

def answer_question_cte(
    context: str,
    question: str,
) -> QuestionAnsweringResultCTE: 
    result = _answer_question_cte(context, question)
    return QuestionAnsweringResultCTE(triplets=result.triplets, answer=result.answer)

In [ ]:
#|export

SYSTEM_PROMPT_CTE_COT = """You are an excellent question-answering system known for providing accurate and reliable answers. Your responses should be solely based on context the information given, without drawing on prior knowledge. Always provide clear and logical step-by-step reasoning in your response.

Before answering the question, first, you extract relevant entity-relation-entity triplets from the context. Then, you answer the question based on the triplets. For instance, 

# Example
Context: "Glenhis Hernández (born 7 October 1990 in Havana) is a taekwondo practitioner from Cuba. She was the 2013 World
Champion in middleweight.

The current mayor of Havana ("President of the People's Power Provincial Assembly") is Marta Hernández Romero, she
was elected on March 5, 2011."

Question: "Who is the current mayor of city Glenhis Hernández was born?"

Triplets: "Glenhis Hernández (Athlete) | born on | October 7, 1990
Glenhis Hernández (Athlete) | birth place | Havana
Glenhis Hernández (Athlete) | specializes in | taekwondo
Glenhis Hernández (Athlete) | won | 2013 World Champion title (Middleweight)
Marta Hernández Romero (Politician) | serves as | mayor of Havana
Marta Hernández Romero (Politician) | holds | the position of 'President of the People's Power Provincial Assembly'
Marta Hernández Romero (Politician) | elected on | March 5, 2011."

Reasoning: "Glenhis Hernández (Athlete) | birth place | Havana
This indicates that Glenhis Hernández was born in Havana.

Marta Hernández Romero (Politician) | serves as | mayor of Havana
This states that Marta Hernández Romero is the mayor of Havana.

From these triplets, we conclude that Marta Hernández Romero is the mayor of Havana."

Answer: "Marta Hernández Romero"
"""

class QuestionAnsweringResultCTECOT(BaseModel):
    """Data model for answering the question."""

    triplets: list[str] = Field(description="A list of entity-relation-entity triplets extracted from the context.")
    reasoning: str = Field(description="Step-by-step reasoning for the answer.")
    answer: str = Field(description="The answer to the question in 2-4 words.")


@magentic.chatprompt(
    magentic.SystemMessage(SYSTEM_PROMPT_CTE_COT), 
    magentic.UserMessage(USER_PROMPT),
)
def answer_question_cte_cot(
    context: str,
    question: str,
) -> QuestionAnsweringResultCTECOT: ...

In [ ]:
context = """Sings America is an album released by David Hasselhoff in August 2004 (see 2004 in music). The album contains covers of songs originally made famous by artists such as Elvis Presley, The Beach Boys, Glen Campbell, Burt Bacharach and Madonna. The German release contains a bonus track, "More Than Words Can Say", which is the only original Hasselhoff composition on the album (written in conjunction with Wade Hubbard and Glenn Morrow)."""
question = "Who performed Sings America?"
answer = "David Hasselhoff"

In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1):
    result = answer_question_standard(context=context, question=question)
print(result.answer)

David Hasselhoff


In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1):
    result = answer_question_cot(context=context, question=question)
print(result.answer)
print()
print(result.reasoning)

David Hasselhoff

The album 'Sings America' was released by David Hasselhoff in August 2004, as mentioned in the context information.


In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1):
    result = answer_question_cot_fs(context=context, question=question)
print(result.answer)
print()
print(result.reasoning)

David Hasselhoff

The context mentions that the album 'Sings America' was released by David Hasselhoff in August 2004. It contains covers of songs originally made famous by various artists, including Elvis Presley, The Beach Boys, Glen Campbell, Burt Bacharach, and Madonna. Additionally, the German release of the album includes a bonus track, 'More Than Words Can Say', which is the only original composition by David Hasselhoff.


In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=1.0):
    result = answer_question_cot_fs(context=context, question=question)
print(result.answer)
print()
print(result.reasoning)

David Hasselhoff

The context information states that the album 'Sings America' was released by David Hasselhoff in August 2004. It mentions the covers of songs by various artists on the album, as well as the bonus track 'More Than Words Can Say,' which is an original Hasselhoff composition.


In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1):
    result = answer_question_cte(context=context, question=question)
print(result.answer)
print()
for triplet in result.triplets:
    print(triplet)
print()
print(result.reasoning)

David Hasselhoff

David Hasselhoff | released | Sings America




In [ ]:
with magentic.OpenaiChatModel("gpt-3.5-turbo", temperature=0.1):
    result = answer_question_cte_cot(context=context, question=question)
print(result.answer)
print()
for triplet in result.triplets:
    print(triplet)
print()
print(result.reasoning)

David Hasselhoff

David Hasselhoff (Musician) | released | Sings America album
David Hasselhoff (Musician) | released in | August 2004
David Hasselhoff (Musician) | album contains | covers of songs by various artists
David Hasselhoff (Musician) | album contains | bonus track 'More Than Words Can Say'
David Hasselhoff (Musician) | composed | 'More Than Words Can Say' with Wade Hubbard and Glenn Morrow

David Hasselhoff (Musician) | released | Sings America album
This indicates that David Hasselhoff performed Sings America album.

From these triplets, we can conclude that David Hasselhoff performed Sings America.


In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()